In [4]:
# 抓取嵌套网页的title/对跳转页面进行解密
from urllib import parse
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import requests
import urllib.request 
import urllib.error
from requests.exceptions import ReadTimeout,HTTPError,RequestException


from html.parser import HTMLParser
def decodeHtml(input):
    h = HTMLParser()
    s = h.unescape(input)
    return s

def get_formula(URL_HASH):
    #URL加密公式部分
    pattern_formula=re.compile("a\[\d\]=\"([a-zA-Z0-9\/:\?=.]+)\";")
    formula=re.findall(pattern_formula,URL_HASH)
    return(formula)

def get_rp_list_new(URL_HASH):
    #URL加密因子部分（源字符串筛选分割）
    pattern_factor=re.compile(",\'([a-zA-Z0-9\|]+)\'.split\(\'\|\'\),")
    result2=re.findall(pattern_factor,URL_HASH)
    result2_list=result2[-1].split('|')
    #print(result2_list)
    
    #URL加密因子部分（制作0-9，a-z，A-Z）
    rp_list=[]
    j=48
    while j<=122:
        if  j>=48 and j<=57:
            rp_list.append(chr(j))
            j+=1
        elif j>=65 and j<=90:
            rp_list.append(chr(j).lower())
            j+=1
        elif j>=97 and j<=122:
            rp_list.append(chr(j).upper())
            j+=1
        else:
            j+=1

    #URL加密因子部分（并入对应值）
    k=0
    for i in result2_list:
        if k<len(result2_list):
            rp_list[k]=[rp_list[k],i]
            k+=1

    #URL加密因子部分（删除多余的）        
    m=len(result2_list)
    while m<len(rp_list):
        del(rp_list[m])

    #URL加密因子部分（删除空的）
    rp_list_new=[]
    for n in rp_list:
        if len(n[1])>0:
            rp_list_new.append(n)
        else:
            pass
    return(rp_list_new)

def formula_unescape(url_orig,rp_list_new):
    address=[]
    for i in url_orig:
        for [u,v] in rp_list_new:
            if u==i:
                i=i.replace(u,v)
                break
            else:
                pass
        address.append(i)
    return(''.join(address))

def hashurl_unescape(url,URL_HASH):
    formula=get_formula(URL_HASH)
    rp_list_new=get_rp_list_new(URL_HASH)
    html2=[]
    for i in formula:
        html2.append([url,''.join(formula_unescape(i,rp_list_new))])
    return(html2)



def getHtml(url):
    req = requests.get(url=url)
    req.encoding = None  
    soup = BeautifulSoup(req.text,'lxml')
    #html33=[]
    for a in soup.find_all('script'):
        html1=str(a.text)
        #html2=hashurl_unescape(url,html1)
        #html33=html33+html2
    return (html1)


def getSkipWeb1(url):
    req = requests.get(url=url)
    req.encoding = None  
    soup = BeautifulSoup(req.text,'lxml')
    a=soup.find_all('meta')
    pattern=re.compile("url=(http:\/\/[0-9a-zA-Z\/:-\?\.]+)")
    output_list=re.findall(pattern,str(a))
    return(output_list)

def getSkipedWeb1(url):
    req = requests.get(url=url)
    req.encoding = None  
    soup = BeautifulSoup(req.text,'lxml')
    a=soup.find_all('script')
    pattern=re.compile("\"(http:\/\/[0-9a-zA-Z\/:-\?\.]+)\";")
    output_list=re.findall(pattern,str(a))
    return(output_list)


from html.parser import HTMLParser
def decodeHtml(input):
    h = HTMLParser()
    s = h.unescape(input)
    return s

In [5]:
# 判断原始网页是否能打开，增加原始网页的模拟关闭，判断链接网页是否能打开，对title部分是跳转的网页进行处理
# 调用chrome浏览器并后台运行
#option=webdriver.ChromeOptions()
#option.add_argument('headless')
#driver = webdriver.Chrome(options=option)

#url_origin=['http://www.cqfmlkj.com/','http://www.weidicar.com/bzh.php','http://www.gjhuali.com/',
#    'http://www.hbltfm.com/index.html','http://www.mmzaojiao.com/','http://www.milaile.com/',
#     'http://www.taoyiezu.com/','http://www.czcin.com/',' https://www.gdyiyue.com/',
#    'https://www.szhouai.com/','http://www.hybet.net/#index','http://www.supaiw.com/',
#    'lt127.com','yzc265.com','www.lt2228.com','www.lt076.com','https://pts227.com',
#    'https://www.wofacai888.com/sc','www.itb999.com','www.vic308.com','js8281.com',
#    '493.com','www.buyu9999.com','28836.com','letou.com','dhy1119.com','https://www.fulibet88.com/',
#    'https://www.toe9570.com/#/register?agentCode=1Ef0P02l','https://888.qq8p.top/',
#    'http://www.lhtpxm.icu/','http://www.isheng360.com/db.php','http://www.38331155.com:8888/?a=10295124',
#    'http://www.44009908.com/','http://www.rdnxqvk.cn/',
#    'http://www.jitaba.cn/applkX/Kw/','http://www.qt211.com/mip/cO26CH.html',
#    'http://51fa51.com/','http://103.194.186.43:9966/','http://www.ufo-1.cn/b6xuf8/7197084833.html',
#    'https://cszx1.com/','http://bcfc5880.vip','321365.com','dsndsn168.com',
#     '53618a.com','921613.com','1889tyc.com','077qm.com','https://www.sk500.cn',
#     '9192655.com','http://m.yihaocai55.com/home','7606.com','www.2348.cc',
#     'b2b.louzhu520.com/','https://www.jitatang.com/paogjcym.html',' http://home.slieny.com/vipbkll/',
#     'www.wtobag.com/hot3n6/','https://www.chalook.net/vipqchh/','http://md.itlun.cn/hot-CYUae9Hl.html',
#     'http://www.cc222.com/FaUP8FAjGB.xml','http://www.txahz.com/dst7G3SlH.html']

url_origin=['http://www.weidicar.com','http://www.gjhuali.com/',
    'http://www.mmzaojiao.com/','http://www.milaile.com/',
    'http://www.taoyiezu.com/','http://www.czcin.com/',
    'http://www.hybet.net/#index',
    'lt127.com','yzc265.com','www.lt2228.com','www.lt076.com','https://pts227.com',
    'www.itb999.com','www.vic308.com','js8281.com',
    '493.com','www.buyu9999.com','28836.com','letou.com','dhy1119.com','https://www.fulibet88.com/',
    'https://www.toe9570.com/#/register?agentCode=1Ef0P02l','https://888.qq8p.top/',
    'http://www.isheng360.com/db.php','http://www.38331155.com:8888/?a=10295124',
    'http://www.44009908.com/','http://www.rdnxqvk.cn/',
    'http://www.jitaba.cn/applkX/Kw/','http://www.qt211.com/mip/cO26CH.html',
    'http://51fa51.com/','http://103.194.186.43:9966/','http://www.ufo-1.cn/b6xuf8/7197084833.html',
    'https://cszx1.com/','http://bcfc5880.vip','321365.com','dsndsn168.com',
     '53618a.com','921613.com','1889tyc.com','077qm.com','https://www.sk500.cn',
     '9192655.com','http://m.yihaocai55.com/home','7606.com','www.2348.cc',
     'b2b.louzhu520.com/','https://www.jitatang.com/paogjcym.html',' http://home.slieny.com/vipbkll/',
     'www.wtobag.com/hot3n6/','https://www.chalook.net/vipqchh/','http://md.itlun.cn/hot-CYUae9Hl.html',
     'http://www.cc222.com/FaUP8FAjGB.xml','http://www.txahz.com/dst7G3SlH.html']

data=[]
for url_old in url_origin:
    y=0
    while y<5:
        try:
            option=webdriver.ChromeOptions()
            option.add_argument('headless')
            driver = webdriver.Chrome(options=option)
            driver.get(url_old)
            driver.set_page_load_timeout(15)
            time.sleep(15)
            #print(driver.current_url)
            #print('-----------------------------------------------------------------------------------------------------------------')
            iframe_name = driver.find_element_by_tag_name('iframe').get_attribute("src")
            iframe = driver.find_elements_by_tag_name('iframe')
            #print(iframe_name)
            driver.switch_to.frame(iframe[0])                          # 最重要的一步
            soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.close()
            for a in soup.find_all('a'):
                    link = a['href']
                    url = parse.urljoin(iframe_name, link)
        except:
            url = url_old            
        print(url)   
        print('------------------------------------------------------------------------------------------')
            try:
                req = requests.get(url)
                req.encoding = None  
        
                soup = BeautifulSoup(decodeHtml(req.text),'lxml')
                if (soup.find_all("title")==[] or str(soup.find_all("title"))=='[<title></title>]' or str(soup.find_all("title"))=='[<title>正在进入</title>]') and soup.find_all("meta")!=[] and getHtml(url).find('eval(function(p,a,c,k,e,d)')<=0:
                    try:
                        ScanSkipAddrList=getSkipWeb1(url)
                        for ScanSkipAddr in ScanSkipAddrList:
                            SkipedURL=getSkipedWeb1(ScanSkipAddr)
                            for i in SkipedURL:
                                data.extend([[url_old,url,i]])
                    except:
                        data.extend([[url_old,url,"人工判断1"]])
                elif (soup.find_all("title")==[] or str(soup.find_all("title"))=='[<title></title>]' or str(soup.find_all("title"))=='[<title>正在进入</title>]')  and getHtml(url).find('eval(function(p,a,c,k,e,d)')>0:
                    try:
                        html2=hashurl_unescape(url,getHtml(url))
                        data.extend([[url_old,html2]])
                    except:
                        data.extend([[url_old,url,"人工判断3"]])
                elif (soup.find_all("title")!=[] and str(soup.find_all("title"))!='[<title></title>]' and str(soup.find_all("title"))!='[<title>正在进入</title>]'):
                    try:
                        data.extend([[url_old,url,''.join(str(soup.find_all('title')))]])
                    except:
                        data.extend([[url_old,url,"人工判断4"]])  
                else :
                    z=0
                    while z<5:
                        try:
                            option=webdriver.ChromeOptions()
                            option.add_argument('headless')
                            driver = webdriver.Chrome(options=option)
                            #driver = webdriver.Chrome()
                            driver.get(url)
                            driver.set_page_load_timeout(11)
                            time.sleep(10)
                            data.extend([[url_old,url,driver.current_url]])
                            driver.close()
                            z+=1
                        except:
                            data.extend([[url_old,url,"人工判断5"]])
                            z+=1 
                y+=1
            except:
                data.extend([[url_old,"人工判断22",'kong']])
                y+=1



IndentationError: unexpected indent (<ipython-input-5-c84e2cbbca67>, line 71)

In [ ]:
import numpy
import pandas
#print(data)
data=numpy.array(data)
df=pandas.DataFrame(data)
df.head()
#print(df1.shape)

In [ ]:
import numpy
import pandas
#print(data)
data=numpy.array(data)
df=pandas.DataFrame(data)
print(df.columns.values)

In [ ]:
print(df)

In [ ]:
df1=df.drop_duplicates(subset=[1,2], keep='first', inplace=False)
df1.to_csv("a0112.csv")